In [1]:
import pandas as pd
import numpy as np
import pickle
import warnings
import copy
import random
import matplotlib.pyplot as plt
from collections import Counter
import joblib
import secrets

In [2]:
df_all = pd.read_csv("portscan_test.csv")

In [3]:
df_all['Label'].value_counts()

Benign    163877
ddos      163877
Name: Label, dtype: int64

In [4]:
## Specify the attack type
attack_type = 'Port Scan'

In [5]:
## Specify the agent to be used
trained_agent = "Agent_DQN_DDoS_ensemble.pkl"

In [6]:
## Number of testing iterations and number of tries given to the agent
num_episodes = 2000
episode_length = 30

In [7]:
## Specify the classifier to be used against the agent
rfc_binary = joblib.load("Port_DNN.joblib")

In [8]:
# rfc_binary.predict(state_extractor(df_all.iloc[100]))

In [9]:
df = df_all[df_all['Label'] == attack_type]

In [10]:
df.drop(columns = ["Label"], axis=1, inplace=True)

C:\Users\diwaspaudel\AppData\Local\Temp\ipykernel_2468\1338034999.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns = ["Label"], axis=1, inplace=True)


In [11]:
df = df[[ 'eth_header', 'ip_header', 'tcp_header','tcp_options', 'tcp_segment_data']]

In [12]:
columns = [ 'eth_header', 'ip_header', 'tcp_header','tcp_options', 'tcp_segment_data']

In [13]:
df['all_payload'] = df['eth_header'] + df['ip_header'] + df['tcp_header'] + df['tcp_options'] + df['tcp_segment_data']

In [14]:
df

,eth_header,ip_header,tcp_header,tcp_options,tcp_segment_data,all_payload
2,029e424890aa02ca69c46d060800,45000028557540007f06bc5912dae5ebac1f451c,e3550050007618e062ae45fe50117fe2a0470000,00,00,029e424890aa02ca69c46d06080045000028557540007f...
3,029e424890aa02ca69c46d060800,450000281c9d40007f06f53112dae5ebac1f451c,cbe20050f9c9a86ec560821450117fe2900f0000,00,00,029e424890aa02ca69c46d060800450000281c9d40007f...
4,029e424890aa02ca69c46d060800,450200341cd040007f06f4f012dae5ebac1f451c,f2080050af63c70a0000000080c2ffff1b8b0000,020405b40103030501010402,00,029e424890aa02ca69c46d060800450200341cd040007f...
7,029e424890aa02ca69c46d060800,450000286dcb40007f06a40312dae5ebac1f451c,db9300501a51cf094b55cf855010800065b90000,00,00,029e424890aa02ca69c46d060800450000286dcb40007f...
8,029e424890aa02ca69c46d060800,4500013355f040007f06bad312dae5ebac1f451c,ca260050b6b3722fd20678b850188000b6c90000,00,474554202f20485454502f312e300d0a4163636570743a...,029e424890aa02ca69c46d0608004500013355f040007f...
...,...,...,...,...,...,...
327746,029e424890aa02ca69c46d060800,45000028162840007f06fba612dae5ebac1f451c,f04f0050cb40d28554b0f94e50107fe2698b0000,00,00,029e424890aa02ca69c46d06080045000028162840007f...
327747,029e424890aa02ca69c46d060800,45000028347c40007f06dd5212dae5ebac1f451c,dd190050299a5ecaf367938350117fe259360000,00,00,029e424890aa02ca69c46d06080045000028347c40007f...
327751,029e424890aa02ca69c46d060800,45000028443840007f06cd9612dae5ebac1f451c,ece60050f98c69dbf2fbea505010800017e70000,00,00,029e424890aa02ca69c46d06080045000028443840007f...
327752,029e424890aa02ca69c46d060800,450000287ede40007f0692f012dae5ebac1f451c,f849005087fdc239c10abc5450107fe285c00000,00,00,029e424890aa02ca69c46d060800450000287ede40007f...


In [15]:
## Actions with discrete Choices

## Action 1: Fragmentation - ip header 2, 3(length),  ip header 10, 11 (checksum), ip header byte 6 and 7 (fragmentation)
             #Time To Live field value is reduced by one (ip_header_byte_8) #https://packetpushers.net/ip-fragmentation-in-detail/
             #more fragement can also be on (20 hex instead of 40) but the second byte ip header byte 7 should be always 00(no change imean).
        
## Action 2: Time to live (ip_header_byte_8) also the checksum bytes

## Action 3: TCP Window size : 'tcp_header_byte_14', 'tcp_header_byte_15'
## tcp options MSS >>tcp option byte 2,and 3 decrease by small amount

## window scaling factor (from 0 to 14) tcp options byte 

## Action 4: TCP options NOP: 

## Action 5: addition of payloads: affects 'ip_header_byte_2', 'ip_header_byte_3': total length of packet
                 #####'ip_header_byte_10', 'ip_header_byte_11': ip header checksum (2 bytes)
                 #####'tcp_header_byte_16', 'tcp_header_byte_17' : tcp checksum

In [16]:
payload_1 = '474554202f3f785857333d6b646872646552622668575479446c445054723d436b596f374b266565506879713d4354677171533356367750574d6247266e506b4a633d446c78413772676c734362426720485454502f312e310d0a4163636570742d456e636f64696e673a202a2c20677a69700d0a486f73743a203230352e3137342e3136352e36380d0a4b6565702d416c6976653a203536340d0a4163636570742d436861727365743a207574662d382c49534f2d383835392d313b713d302e302c2a3b713d302e390d0a436f6e6e656374696f6e3a206b6565702d616c6976650d0a43616368652d436f6e74726f6c3a206d61782d6167653d300d0a557365722d4167656e743a204d6f7a696c6c612f352e30202857696e646f7773204e5420362e333b20574f57363429204170706c655765624b69742f3533352e333620284b48544d4c2c206c696b65204765636b6f292056657273696f6e2f352e302e34205361666172692f3533372e31300d0a436f6e74656e742d547970653a206170706c69636174696f6e2f782d75726c2d656e636f6465640d0a0d0a'
payload_2 = '474554202f3f33616965376267383d4d305232414626746837414e423d714f516631717664746120485454502f312e310d0a486f73743a203230352e3137342e3136352e36380d0a4163636570742d456e636f64696e673a20677a69700d0a4b6565702d416c6976653a203638310d0a557365722d4167656e743a204d6f7a696c6c612f352e30202857696e646f7773204e5420362e313b20574f57363429204765636b6f2f32303035323531312046697265666f782f31332e300d0a4163636570742d436861727365743a2049534f2d383835392d322c7574662d383b713d302e312c2a3b713d302e330d0a436f6e6e656374696f6e3a206b6565702d616c6976650d0a526566657265723a20687474703a2f2f7777772e79616e6465782e636f6d2f774a46776b733f676f52423d4b4a320d0a43616368652d436f6e74726f6c3a206e6f2d63616368650d0a436f6e74656e742d547970653a206170706c69636174696f6e2f782d75726c2d656e636f6465640d0a0d0a'
payload_3 = '474554202f3f646d62326f67627935503d4831336d3550684a33697563565851266947466734333d4d5331655837694746756e504555624a556820485454502f312e310d0a4163636570742d456e636f64696e673a206964656e746974792c20677a69700d0a436f6e6e656374696f6e3a206b6565702d616c6976650d0a4b6565702d416c6976653a203435350d0a557365722d4167656e743a204d6f7a696c6c612f352e30202857696e646f77733b20553b204d53494520362e313b2057696e646f7773204e5420362e313b202e4e455420434c5220322e352e393137393b2057696e36343b20783634290d0a4163636570742d436861727365743a207574662d382c49534f2d383835392d313b713d302e332c2a3b713d302e370d0a486f73743a203230352e3137342e3136352e36380d0a526566657265723a20687474703a2f2f7777772e62696e672e636f6d2f4456656e460d0a43616368652d436f6e74726f6c3a206e6f2d63616368650d0a436f6e74656e742d547970653a206170706c69636174696f6e2f782d75726c2d656e636f6465640d0a0d0a'

In [17]:
def transformed_hex_preprocess(decimal):
    
    hexa = hex(decimal).lstrip("0x")
    if (len(hexa) % 2) != 0:
        hexa = '0' + hexa
    return hexa

In [18]:
def hex_norm(payload, col):
    dict_hex_cols = {'ip_header':20 , 'tcp_header': 20 , 'tcp_options': 40, 'tcp_segment_data': 1460 }
    byte_array = bytes.fromhex(payload)
    byte_lst = list(byte_array)
    if (len(byte_lst) < dict_hex_cols[col]):
            output = np.pad(byte_lst, (0, dict_hex_cols[col]-len(byte_lst)), 'constant')
    else:
            output = np.array(byte_lst[0:dict_hex_cols[col]].copy())
            
    #normalize       
    output = np.abs(output.astype(float)) / 255
    return output

In [19]:
def concat_dfs(df1, df2, df3, df4):
    packet_p = pd.DataFrame([df1]).add_prefix('ip_header_byte_')
    packet_t = pd.DataFrame([df2]).add_prefix('tcp_header_byte_')
    packet_n = pd.DataFrame([df3]).add_prefix('tcp_options_byte_')
    packet_x = pd.DataFrame([df4]).add_prefix('tcp_segment_data_byte_')
    
    packet_purturbed = pd.concat([packet_p , packet_t , packet_n, packet_x ], axis =1)
    
    return packet_purturbed

In [20]:
def df_preprocess(df):
    
    df.drop(columns=['ip_header_byte_9','ip_header_byte_0', 'ip_header_byte_1'],inplace=True) #remove ipv4 byte, differentiated service filed, and protocol bytes
    packet_purturbed_final = df.drop(df.loc[:, 'ip_header_byte_12':'tcp_header_byte_3'].columns, axis=1) #remove src and dest ips and ports bytes
    return packet_purturbed_final

In [21]:
def calc_checksum(packet):
    pkt = copy.deepcopy(packet)
    checksum_header = pkt['ip_header'][:20] + pkt['ip_header'][24:]
    sum_ = 0
    for i in range(0,len(checksum_header),4):
        h = checksum_header[i] + checksum_header[i+1] + checksum_header[i+2] + checksum_header[i+3]
        sum_ += int(h,16)
    
    ch = hex(sum_).lstrip("0x")

    if len(ch) % 4 != 0:
        ch_ = '000'+ ch
    
    check_ = int(ch_[:4],16) + int(ch_[4:],16)
    check_hex = hex(check_).lstrip("0x")

    if len(check_hex) > 4:
        for i in range(8-len(check_hex)):
            check_hex = '0' + check_hex
        check_hex = transformed_hex_preprocess(int(check_hex[:4],16) + int(check_hex[4:],16))

    check_sum_dec = int('ffff',16) - int(check_hex,16)

    check_sum = hex(check_sum_dec).lstrip("0x")
    
    if check_sum_dec != 0:
        if len(check_sum) % 4 != 0:
            for d in range(4-len(check_sum)%4):
                check_sum = '0' + check_sum
    else:
        check_sum = '0000'

    return check_sum

In [22]:
# https://www.securitynik.com/2015/08/calculating-udp-checksum-with-taste-of_3.html

In [23]:
def calc_tcp_checksum(packet):
    pkt = copy.deepcopy(packet)
    tcp_checksum_header = pkt['tcp_header'][0:32] + pkt['tcp_header'][36:]
    sum_header = 0
    for i in range(0,len(tcp_checksum_header),4):
        h1 = tcp_checksum_header[i] + tcp_checksum_header[i+1] + tcp_checksum_header[i+2] + tcp_checksum_header[i+3]
        sum_header += int(h1,16)
    tcp_options = pkt['tcp_options']
    if len(tcp_options) % 4 != 0:
        for i in range(4-len(tcp_options) % 4):
            tcp_options = '0' + tcp_options
    sum_options = 0
    for i in range(0,len(tcp_options),4):
        h2 = tcp_options[i] + tcp_options[i+1] + tcp_options[i+2] + tcp_options[i+3]
        sum_options += int(h2,16)
    
    tcp_payload = pkt['tcp_segment_data']
    if len(tcp_payload) % 4 != 0:
        int_fg = len(tcp_payload)// 4
        pad = ''
        for a in range(4 - len(tcp_payload) % 4):
            pad += '0'
        tcp_payload = tcp_payload[: int_fg*4] + pad + tcp_payload[int_fg*4:]

    
    sum_payload = 0
    for i in range(0,len(tcp_payload),4):
        h3 = tcp_payload[i] + tcp_payload[i+1] + tcp_payload[i+2] + tcp_payload[i+3]
        sum_payload += int(h3,16)
        
    if len(pkt['tcp_segment_data']) <= 2:
        tcp_segment_len = 0
    elif len(pkt['tcp_segment_data']) >= 2920:
        tcp_segment_len = 2920 // 2
    else:
        tcp_segment_len = len(pkt['tcp_segment_data'])// 2

    if len(pkt['tcp_options']) <= 2:
        tcp_options_len = 0
    else:
        tcp_options_len = len(pkt['tcp_options'])// 2
    
    
    
    total_length_ = int(pkt['ip_header'][4:8],16) - 20 
    
    
    tcp_check_dec = int(pkt['ip_header'][24:28],16) + int(pkt['ip_header'][28:32],16) + int(pkt['ip_header'][32:36],16)\
            + int(pkt['ip_header'][36:],16)  + total_length_ + sum_header + sum_options+ sum_payload + 6 
    
    
    ch = hex(tcp_check_dec).lstrip("0x")
    
    if len(ch) % 4 != 0:
        for j in range(4-len(ch)%4):
            ch = '0'+ ch
    
        
    check_ =  int(ch[4:],16) + int(ch[:4],16)
    check_hex = hex(check_).lstrip("0x")
    if len(check_hex) > 4:
        for i in range(8-len(check_hex)):
            check_hex = '0' + check_hex
        check_hex = transformed_hex_preprocess(int(check_hex[:4],16) + int(check_hex[4:],16))

    check_sum_dec = int('ffff',16) - int(check_hex,16)
    
    check_sum = hex(check_sum_dec).lstrip("0x")
    
    if check_sum_dec > 0:
        if len(check_sum) % 4 != 0:
            for d in range(4-len(check_sum)%4):
                check_sum = '0' + check_sum
    else:
        check_sum = '0000'
    return check_sum

In [24]:
## Action 1
def action_frag(pkt, payload):
    packet = copy.deepcopy(pkt)
    #frag is allowed ()
    packet['ip_header'] = packet['ip_header'][:12] + "00" + packet['ip_header'][14:]
    
    #change in length
    added_length = len(payload)//2 
    old_length_dec = int(packet['ip_header'][4:8], 16)
    new_length_dec = added_length + old_length_dec
    new_length_hex = transformed_hex_preprocess(new_length_dec)
    if len(new_length_hex) < 4 :
        new_length_hex  = '00'+ new_length_hex
    
    packet['ip_header']=packet['ip_header'][:4] + new_length_hex + packet['ip_header'][8:]
    
    #reduce ttl by 1
    old_ttl_dec = int(packet['ip_header'][16:18], 16)
    new_ttl_dec = old_ttl_dec - 1
    if new_ttl_dec <= 1:
        new_ttl_dec = 1
    new_ttl_hex = transformed_hex_preprocess(new_ttl_dec)
    packet['ip_header']=packet['ip_header'][:16] + new_ttl_hex + packet['ip_header'][18:]
    
    #IP checksum update
    new_checksum_hex = calc_checksum(packet)
    packet['ip_header']=packet['ip_header'][:20] + new_checksum_hex + packet['ip_header'][24:]

    #segment data update
    if packet['tcp_segment_data'] == '00':
         packet['tcp_segment_data']= payload
            
    else:
         packet['tcp_segment_data']=packet['tcp_segment_data'] + payload
            
    ## TCP Checksum update
    new_tcp_checksum_hex = calc_tcp_checksum(packet)
    packet['tcp_header'] = packet['tcp_header'][0:32] + new_tcp_checksum_hex + packet['tcp_header'][36:]

    return packet

In [25]:
## Action 2
def action_more_frag(pkt, payload):
    packet = copy.deepcopy(pkt)
    #frag is allowed ()
    packet['ip_header']= packet['ip_header'][:12] + "20" + packet['ip_header'][14:]
    
    #change in length
    added_length = len(payload)//2 
    old_length_dec = int(packet['ip_header'][4:8], 16)
    new_length_dec = added_length + old_length_dec
    new_length_hex = transformed_hex_preprocess(new_length_dec)
    if len(new_length_hex) < 4 :
        new_length_hex  = '00'+ new_length_hex
    packet['ip_header']=packet['ip_header'][:4] + new_length_hex + packet['ip_header'][8:]
    
    #reduce ttl by 1
    old_ttl_dec = int(packet['ip_header'][16:18], 16)
#     print(old_ttl_dec)

    new_ttl_dec = old_ttl_dec - 1
    if new_ttl_dec <= 1:
        new_ttl_dec = 1
        
#     print(new_ttl_dec)
    new_ttl_hex = transformed_hex_preprocess(new_ttl_dec)
#     print('new_ttl_hex',new_ttl_hex)
    packet['ip_header']=packet['ip_header'][:16] + new_ttl_hex + packet['ip_header'][18:]
    
#     print(len(packet['ip_header']))
    #IP checksum update
    new_checksum_hex = calc_checksum(packet)
#     print('new_checksum_hex',new_checksum_hex)
    packet['ip_header']=packet['ip_header'][:20] + new_checksum_hex + packet['ip_header'][24:]

    #segment data update
    if packet['tcp_segment_data'] == '00':
         packet['tcp_segment_data']= payload
            
    else:
         packet['tcp_segment_data']=packet['tcp_segment_data'] + payload
            
    ## TCP Checksum update
    new_tcp_checksum_hex = calc_tcp_checksum(packet)
    packet['tcp_header'] = packet['tcp_header'][0:32] + new_tcp_checksum_hex + packet['tcp_header'][36:]

    return packet

In [26]:
## Action 3
def action_ttl(pkt,change_amount):
    packet = copy.deepcopy(pkt)
    
    #change ttl by change amount
    old_ttl_dec = int(packet['ip_header'][16:18], 16)
    new_ttl_dec = old_ttl_dec + change_amount
    
    if new_ttl_dec > 255:
        new_ttl_dec = 255
    elif new_ttl_dec <= 0:
        new_ttl_dec = 1
    new_ttl_hex = transformed_hex_preprocess(new_ttl_dec)
    packet['ip_header']=packet['ip_header'][:16] + new_ttl_hex + packet['ip_header'][18:]
    
    #IP checksum update
    new_checksum_hex = calc_checksum(packet)
    packet['ip_header']=packet['ip_header'][:20] + new_checksum_hex + packet['ip_header'][24:]
    
    return packet

In [27]:
## Action 4
def action_window_size(pkt,change_amount):
    packet = copy.deepcopy(pkt)
    
    # Change window size by change amount
    old_window_dec = int(packet['tcp_header'][28:32],16)
    new_window_dec = old_window_dec + change_amount
    
    if new_window_dec > 65535:
        new_window_dec = 65535
    elif new_window_dec <= 0:
        new_window_dec = 1
        
    new_window_hex  = transformed_hex_preprocess(new_window_dec)
    
    
    if len(new_window_hex) < 4:
        for i in range(4 - len(new_window_hex)):
            new_window_hex  = '0' + new_window_hex

    packet['tcp_header'] = packet['tcp_header'][:28] + new_window_hex + packet['tcp_header'][32:]
    
    # TCP Checksum update
    new_tcp_checksum_hex = calc_tcp_checksum(packet)
    packet['tcp_header'] = packet['tcp_header'][0:32] + new_tcp_checksum_hex + packet['tcp_header'][36:]
    
    return packet

In [28]:
## Action 5
def action_add_payload(pkt,payload):
    packet = copy.deepcopy(pkt)

    #segment data update
    if packet['tcp_segment_data'] == '00':
         packet['tcp_segment_data']= payload
            
    else:
         packet['tcp_segment_data']=packet['tcp_segment_data'] + payload
            
    # IP header total length change 
    added_length = len(payload)//2 
    old_length_dec = int(packet['ip_header'][4:8], 16)
    new_length_dec = added_length + old_length_dec
    new_length_hex = transformed_hex_preprocess(new_length_dec)
    if len(new_length_hex) < 4 :
        new_length_hex  = '00'+ new_length_hex
    packet['ip_header']=packet['ip_header'][:4] + new_length_hex + packet['ip_header'][8:]
   
    # IP Checksum update
    new_checksum_hex = calc_checksum(packet)

    packet['ip_header']=packet['ip_header'][:20] + new_checksum_hex + packet['ip_header'][24:]

    
    ## TCP Checksum update
    new_tcp_checksum_hex = calc_tcp_checksum(packet)
    
    packet['tcp_header'] = packet['tcp_header'][0:32] + new_tcp_checksum_hex + packet['tcp_header'][36:]

    return packet

In [29]:
## Action 6
def action_MSS(pkt,change):
    packet = copy.deepcopy(pkt)
    
    str_ = packet['tcp_options']
    
    
    mss_start_index = str_.find('0204')
    
    
    if mss_start_index != -1:
        og_mss_hex = str_[mss_start_index+4 : mss_start_index+8 ]
    else:
        og_mss_hex = '0000'
    
    
    og_mss_dec = int(og_mss_hex,16)
    new_mss_dec = og_mss_dec + change
    if new_mss_dec < 0:
        new_mss_dec = 5
    
    new_mss_hex = transformed_hex_preprocess(new_mss_dec)
    
    
    if len(new_mss_hex) < 4:
        new_mss_hex = '00' + new_mss_hex
    
        
    if mss_start_index != -1:
        
        str_ = str_[:mss_start_index+4] + new_mss_hex + str_[mss_start_index+8:]
        
        packet['tcp_options'] = str_
        
        ## Change in checksum
        new_tcp_checksum_hex = calc_tcp_checksum(packet)
        packet['tcp_header'] = packet['tcp_header'][0:32] + new_tcp_checksum_hex + packet['tcp_header'][36:]
        
    else:
        str_ = str_ + '01' + '01' + '02' + '04' + new_mss_hex + '01' + '01' 
        
        packet['tcp_options'] = str_
        
        ## Chnage in IP header total length
        added_length = 8
        old_length_dec = int(packet['ip_header'][4:8], 16)
        new_length_dec = added_length + old_length_dec
        new_length_hex = transformed_hex_preprocess(new_length_dec)
        if len(new_length_hex) < 4 :
            new_length_hex  = '00'+ new_length_hex
        packet['ip_header']=packet['ip_header'][:4] + new_length_hex + packet['ip_header'][8:]
        
        ## Change in IP header checksum
        new_checksum_hex = calc_checksum(packet)
        packet['ip_header']=packet['ip_header'][:20] + new_checksum_hex + packet['ip_header'][24:]
        
        ## Change in TCP header length
        old_len_dec = int(packet['tcp_header'][24:26],16)
        new_len_dec = old_len_dec + added_length
        new_len_hex = transformed_hex_preprocess(new_len_dec)
        packet['tcp_header'] = packet['tcp_header'][:24] + new_len_hex + packet['tcp_header'][26:]
        
        ## Change in TCP Checksum
        new_tcp_checksum_hex = calc_tcp_checksum(packet)
        packet['tcp_header'] = packet['tcp_header'][0:32] + new_tcp_checksum_hex + packet['tcp_header'][36:]
    
    return packet

In [30]:
## Action 7
def action_windowscale(pkt,change):
    packet = copy.deepcopy(pkt)
    
    str_ = packet['tcp_options']
    
    
    scale_start_index = str_.find('0303')
    
    
    if scale_start_index != -1:
        og_scale_hex = str_[scale_start_index+4 : scale_start_index+6 ]

    else:
        og_scale_hex = '00'
    
    
    og_scale_dec = int(og_scale_hex,16)
    new_scale_dec = og_scale_dec + change
    if new_scale_dec <= 0:
        new_scale_dec = 1
    
    new_scale_hex = transformed_hex_preprocess(new_scale_dec)


    if scale_start_index != -1:
        
        str_ = str_[:scale_start_index+4] + new_scale_hex + str_[scale_start_index+6:]
        
        packet['tcp_options'] = str_
        
        ## Change in checksum
        new_tcp_checksum_hex = calc_tcp_checksum(packet)
        packet['tcp_header'] = packet['tcp_header'][0:32] + new_tcp_checksum_hex + packet['tcp_header'][36:]
        
    else:
        str_ = str_ + '01' + '01' + '03' + '03' + new_scale_hex + '01' + '01' 
        
        packet['tcp_options'] = str_
        
        ## Chnage in IP header total length
        added_length = 7
        old_length_dec = int(packet['ip_header'][4:8], 16)
        new_length_dec = added_length + old_length_dec
        new_length_hex = transformed_hex_preprocess(new_length_dec)
        if len(new_length_hex) < 4 :
            new_length_hex  = '00'+ new_length_hex
        packet['ip_header']=packet['ip_header'][:4] + new_length_hex + packet['ip_header'][8:]
        
        ## Change in IP header checksum
        new_checksum_hex = calc_checksum(packet)
        packet['ip_header']=packet['ip_header'][:20] + new_checksum_hex + packet['ip_header'][24:]
        
        ## Change in TCP header length
        old_len_dec = int(packet['tcp_header'][24:26],16)
        new_len_dec = old_len_dec + added_length
        new_len_hex = transformed_hex_preprocess(new_len_dec)
        packet['tcp_header'] = packet['tcp_header'][:24] + new_len_hex + packet['tcp_header'][26:]
        
        ## Change in TCP Checksum
        new_tcp_checksum_hex = calc_tcp_checksum(packet)
        packet['tcp_header'] = packet['tcp_header'][0:32] + new_tcp_checksum_hex + packet['tcp_header'][36:]
    
    return packet

In [31]:
random.seed(1)
st_ind = random.randint(1,700)
payload1 = payload_1[st_ind : st_ind+20]

In [32]:
def state_extractor(packet):
    
    ip_header_dec_lst = hex_norm(packet['ip_header'], 'ip_header')
    tcp_header_dec_lst = hex_norm(packet['tcp_header'], 'tcp_header')
    tcp_option_dec_lst = hex_norm(packet['tcp_options'], 'tcp_options')
    tcp_segment_dec_lst = hex_norm(packet['tcp_segment_data'], 'tcp_segment_data')
    
    pkt_df = concat_dfs(ip_header_dec_lst, tcp_header_dec_lst, tcp_option_dec_lst, tcp_segment_dec_lst)
    
    final_pkt_df = df_preprocess(pkt_df)
    
    return final_pkt_df

In [33]:
def syn_check(state):
    syn_ = False
    if state['tcp_header'][26:28] == '02' or state['tcp_header'][26:28] == '12':
        syn_ = True
    return syn_ 

In [34]:
def step(state,action):
    done = False
    ## Fragmentation actions
    ## Frag
    if action == 0:
        next_state = action_frag(state,payload1)
        add_rew = -5
    ## More frag
    elif action == 1:
        next_state = action_more_frag(state,payload1)
        add_rew = -5
        
    ## TIme to live actions
    # Increase TTL
    elif action == 2:
        next_state = action_ttl(state,2)
        add_rew = -1
    # Decrease
    elif action == 3:
        next_state = action_ttl(state,-2)
        add_rew = -1
    
    ## Window Size actions
    # Increase
    elif action == 4:
        next_state = action_window_size(state,20)
        add_rew = -1
    # Decrease
    elif action == 5:
        next_state = action_window_size(state,-20)
        add_rew = -1
    
    ## Maximum segement size actions
    # Increase
    elif action == 6:
        if syn_check(state):
            next_state = action_MSS(state,20)
            add_rew = -5
        else:
            done = True
    # Decrease
    elif action == 7:
        if syn_check(state):
            next_state = action_MSS(state,-20)
            add_rew = -5
        else:
            done = True
    
    ## window scale
    # Increase
    elif action == 8:
        if syn_check(state):
            next_state = action_windowscale(state,2)
            add_rew = -5
        else:
            done = True
    # Decrease
    elif action == 9:
        if syn_check(state):
            next_state = action_windowscale(state,-2)
            add_rew = -5
        else:
            done = True
            
    ## Payload addition
    elif action == 10:
        next_state = action_add_payload(state,payload)
        add_rew = -1
    
#     print(next_state)
    if done != True:
        next_state_df = state_extractor(next_state)
#         print(next_state_df)
        class_pred = rfc_binary.predict(next_state_df)
        warnings.filterwarnings("ignore", category=UserWarning)

        if class_pred[0] != attack_type:
            ## Reward for success
            reward = 600 + add_rew
            done = True
            #next_state = np.array([[1 for i in range(1525)]])

        else:
            ## Reward for failed attempt
            reward = add_rew
            done = False
    else:
        ## Infeasibility Reward
        reward = -1000
        next_state = np.array([[1 for i in range(1525)]])
    return next_state, reward, done

In [35]:
example_ = df.iloc[9]
example_df = state_extractor(example_)
c_n = list(example_df.columns)
ar = np.full((1,1525),1)
final_state_df = pd.DataFrame(ar,columns = c_n)

In [36]:
%matplotlib inline
import math
from collections import namedtuple
from itertools import count
# import tensorflow as tf
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T

In [37]:
## The neural network class with input size = dimension of state space (n_nodes+1), output layer size = 1 (action)

class DQN(nn.Module):
    
    def __init__(self, state_dim):
        super().__init__()
        self.fc1 = nn.Linear(in_features= state_dim , out_features=256)
        self.fc2 = nn.Linear(in_features=256, out_features=128)
        self.fc3 = nn.Linear(in_features=128, out_features=64)
        self.out = nn.Linear(in_features=64, out_features= 11)
    
    def forward(self, t):
        t = t.flatten(start_dim = 1)
        t = F.relu(self.fc1(t))
        t = F.relu(self.fc2(t))
        t = F.relu(self.fc3(t))
        t = self.out(t)
        return t

In [38]:
## Initializing the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [39]:
state_dim = 1525

In [40]:
test_DNN = DQN(state_dim).to(device)
test_DNN.load_state_dict(torch.load(trained_agent))
test_DNN.eval()

DQN(
  (fc1): Linear(in_features=1525, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=64, bias=True)
  (out): Linear(in_features=64, out_features=11, bias=True)
)

In [41]:
df_test_ = pd.read_csv("portscan_test.csv")

In [42]:
# df_test_ = df_test_.dropna(axis = 0, how = "any")
# df_test_ = df_test_.drop(['Unnamed: 0'], axis = 1)
# df_test_['tcp_header'] = df_test_['tcp_header'].apply(lambda x: x + secrets.token_hex(10))

In [43]:
df_test = df_test_[df_test_['Label']!= 'Benign'].reset_index()

In [44]:
df_test['Label'].value_counts()

ddos    163877
Name: Label, dtype: int64

In [45]:
df_test_ddos = df_test[df_test['Label'] == attack_type] ## Change here to whichever attack class you want

In [46]:
df_test_ddos['Label'].value_counts()

ddos    163877
Name: Label, dtype: int64

In [47]:
# num_episodes = len(df_test_infiltration)


In [48]:
num_episodes

2000

In [49]:
df_test_ddos_ = df_test_ddos.drop(columns = ['Label','index'])

In [50]:
df_test_ddos_

,Unnamed: 0,eth_header,ip_header,tcp_header,tcp_options,tcp_segment_data
0,222299,029e424890aa02ca69c46d060800,45000028557540007f06bc5912dae5ebac1f451c,e3550050007618e062ae45fe50117fe2a0470000,00,00
1,979316,029e424890aa02ca69c46d060800,450000281c9d40007f06f53112dae5ebac1f451c,cbe20050f9c9a86ec560821450117fe2900f0000,00,00
2,586513,029e424890aa02ca69c46d060800,450200341cd040007f06f4f012dae5ebac1f451c,f2080050af63c70a0000000080c2ffff1b8b0000,020405b40103030501010402,00
3,365832,029e424890aa02ca69c46d060800,450000286dcb40007f06a40312dae5ebac1f451c,db9300501a51cf094b55cf855010800065b90000,00,00
4,484690,029e424890aa02ca69c46d060800,4500013355f040007f06bad312dae5ebac1f451c,ca260050b6b3722fd20678b850188000b6c90000,00,474554202f20485454502f312e300d0a4163636570743a...
...,...,...,...,...,...,...
163872,255427,029e424890aa02ca69c46d060800,45000028162840007f06fba612dae5ebac1f451c,f04f0050cb40d28554b0f94e50107fe2698b0000,00,00
163873,533098,029e424890aa02ca69c46d060800,45000028347c40007f06dd5212dae5ebac1f451c,dd190050299a5ecaf367938350117fe259360000,00,00
163874,410082,029e424890aa02ca69c46d060800,45000028443840007f06cd9612dae5ebac1f451c,ece60050f98c69dbf2fbea505010800017e70000,00,00
163875,112431,029e424890aa02ca69c46d060800,450000287ede40007f0692f012dae5ebac1f451c,f849005087fdc239c10abc5450107fe285c00000,00,00


In [51]:
adv_examples_df = final_state_df.copy()
original_examples_df = final_state_df.copy()

In [52]:
adv_examples_df = adv_examples_df.drop(0)
original_examples_df  = original_examples_df .drop(0)

In [53]:
import json 
dic_actions = {}
import time
start_time = time.time()

list_win_test_experiences = []
all_test_experiences = []
test_win_count = 0
all_test_epi_rewards = []
print_reward = 0
payload_big = payload_1 + payload_1

for episode in range(num_episodes):
#     print("______________________________________")
#     print("______________________________________")
#     print("Episode",episode)
#     print("______________________________________")
#     print("______________________________________")
    list_experiences = []
    list_actions = []
    state_ = df_test_ddos_.iloc[episode]
    
    original_state_ = df_test_ddos_.iloc[episode]

    state_df = state_extractor(state_)
    state_array = state_extractor(state_).to_numpy()
#     print(state_array)
    class_pred = rfc_binary.predict(state_df)
#     print(class_pred)
    while class_pred != attack_type:
#         print("weird")
        state_ = df_test_ddos_.iloc[episode+1]
        episode += 1

        state_array = state_extractor(state_).to_numpy()
        state_df = state_extractor(state_)
        class_pred = rfc_binary.predict(state_df)
    if episode >= num_episodes:
        break 
    state_tensor = torch.FloatTensor(state_array).to(device)
    payload_no = 0
    epi_reward = 0

            
    for timestep in range(episode_length):
        if episode%500 == 0:
            print("Step",episode,timestep)
#         print("........................")
        transition_list = []
#         print('State', state_array)
        action_ = test_DNN(state_tensor).argmax(dim=1).to(device) 
#         print(action_)
        action = torch.tensor([int(action_)],dtype = torch.int64).to(device)
        list_actions.append(int(action))

        if action_[0] == 10:
            payload_no += 20
            payload = payload_big[payload_no:payload_no+20]
#         print("Action", action)
#         print(action)
        next_state_, reward, done = step(state_,int(action))
        #print(next_state_)
        if reward > 590:
            dic_actions[episode] = list_actions
            adv_examples_df = pd.concat([adv_examples_df, state_extractor(next_state_)], axis=0)
            original_examples_df = pd.concat([original_examples_df, state_extractor(original_state_)], axis=0)
            test_win_count += 1
            
        epi_reward += reward
        if done:
            next_state_array = final_state_df.to_numpy()
        else:
            next_state_array = state_extractor(next_state_).to_numpy()
#         print("Reward",reward)
        next_state_tensor = torch.FloatTensor(next_state_array).to(device)
        
        reward_tensor = torch.FloatTensor([reward]).to(device)
        state_ = next_state_
        if done != True:
            state_array = state_extractor(state_).to_numpy()
    
        state_tensor = next_state_tensor
        
        transition_list.append(state_array)
        transition_list.append(int(action))
        transition_list.append(reward)
        transition_list.append(next_state_array)
        list_experiences.append(transition_list)
        
        if done:
            list_win_test_experiences.append(list_experiences)
            
            break
    
        

    print_reward += epi_reward
    all_test_epi_rewards.append(epi_reward)
    all_test_experiences.append(list_experiences)

end_time = time.time()

Step 0 0
Step 0 1
Step 0 2
Step 0 3
Step 0 4
Step 0 5
Step 0 6
Step 0 7
Step 0 8
Step 0 9
Step 0 10
Step 0 11
Step 0 12
Step 0 13
Step 0 14
Step 0 15
Step 500 0
Step 500 1
Step 500 2
Step 500 3
Step 500 4
Step 500 5
Step 500 6
Step 500 7
Step 500 8
Step 500 9
Step 500 10
Step 500 11
Step 500 12
Step 500 13
Step 500 14
Step 500 15
Step 500 16
Step 500 17
Step 500 18
Step 500 19
Step 500 20
Step 500 21
Step 500 22
Step 500 23
Step 500 24
Step 500 25
Step 500 26
Step 500 27
Step 500 28
Step 500 29
Step 1000 0
Step 1500 0


In [54]:
episode

1999

In [55]:
# Serializing json  
json_object = json.dumps(dic_actions, indent = 4) 


'''
rename the json file appropriately
eg. actions_attackType_classifierType.json
'''


# Writing to sample.json
with open("actions_portscan_DNN.json", "w") as outfile:
    outfile.write(json_object)

In [56]:
# Opening JSON file
with open('actions_portscan_DNN.json', 'r') as openfile:
    # Reading from json file
    json_object = json.load(openfile)

In [57]:
'''
save the data frames appropriately 
eg. adv_examples_attackType_classifierType.csv
eg. original_examples_attackType_classifierType.csv

'''

adv_examples_df.to_csv (r'adv_examples_portscan_DNN.csv', index = None, header=True) 
original_examples_df.to_csv (r'original_examples_portscan_DNN.csv', index = None, header=True) 

In [58]:
episode

1999

In [59]:
len(adv_examples_df)

1358

In [60]:
sum(adv_examples_df.iloc[743].values == original_examples_df.iloc[743].values)

1506

In [61]:
adv_examples_df

,ip_header_byte_2,ip_header_byte_3,ip_header_byte_4,ip_header_byte_5,ip_header_byte_6,ip_header_byte_7,ip_header_byte_8,ip_header_byte_10,ip_header_byte_11,tcp_header_byte_4,...,tcp_segment_data_byte_1450,tcp_segment_data_byte_1451,tcp_segment_data_byte_1452,tcp_segment_data_byte_1453,tcp_segment_data_byte_1454,tcp_segment_data_byte_1455,tcp_segment_data_byte_1456,tcp_segment_data_byte_1457,tcp_segment_data_byte_1458,tcp_segment_data_byte_1459
0,0.0,0.235294,0.333333,0.458824,0.12549,0.0,0.600000,0.760784,0.270588,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,0.0,0.196078,0.109804,0.615686,0.00000,0.0,0.494118,0.211765,0.156863,0.976471,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,0.0,0.282353,0.109804,0.815686,0.00000,0.0,0.490196,0.211765,0.866667,0.686275,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,0.0,0.235294,0.427451,0.796078,0.12549,0.0,0.639216,0.623529,0.937255,0.101961,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,0.0,0.282353,0.466667,0.886275,0.00000,0.0,0.490196,0.858824,0.792157,0.733333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,0.0,0.243137,0.266667,0.537255,0.00000,0.0,0.494118,0.054902,0.180392,0.670588,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,0.0,0.235294,0.239216,0.215686,0.12549,0.0,0.635294,0.819608,0.513725,0.227451,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,0.0,0.196078,0.090196,0.454902,0.00000,0.0,0.494118,0.231373,0.317647,0.278431,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,0.0,0.282353,0.078431,0.654902,0.00000,0.0,0.490196,0.247059,0.023529,0.019608,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [62]:
original_examples_df

,ip_header_byte_2,ip_header_byte_3,ip_header_byte_4,ip_header_byte_5,ip_header_byte_6,ip_header_byte_7,ip_header_byte_8,ip_header_byte_10,ip_header_byte_11,tcp_header_byte_4,...,tcp_segment_data_byte_1450,tcp_segment_data_byte_1451,tcp_segment_data_byte_1452,tcp_segment_data_byte_1453,tcp_segment_data_byte_1454,tcp_segment_data_byte_1455,tcp_segment_data_byte_1456,tcp_segment_data_byte_1457,tcp_segment_data_byte_1458,tcp_segment_data_byte_1459
0,0.0,0.156863,0.333333,0.458824,0.25098,0.0,0.498039,0.737255,0.349020,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,0.0,0.156863,0.109804,0.615686,0.25098,0.0,0.498039,0.960784,0.192157,0.976471,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,0.0,0.203922,0.109804,0.815686,0.25098,0.0,0.498039,0.956863,0.941176,0.686275,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,0.0,0.156863,0.427451,0.796078,0.25098,0.0,0.498039,0.643137,0.011765,0.101961,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,0.0,0.203922,0.466667,0.886275,0.25098,0.0,0.498039,0.600000,0.870588,0.733333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,0.0,0.203922,0.266667,0.537255,0.25098,0.0,0.498039,0.803922,0.215686,0.670588,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,0.0,0.156863,0.239216,0.215686,0.25098,0.0,0.498039,0.831373,0.592157,0.227451,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,0.0,0.156863,0.090196,0.454902,0.25098,0.0,0.498039,0.980392,0.352941,0.278431,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,0.0,0.203922,0.078431,0.654902,0.25098,0.0,0.498039,0.992157,0.098039,0.019608,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
